# Music21 Exploration Template

Use this notebook to explore music21 capabilities, especially rebuilding a score (partition) from a list of notes, and assess suitability for the DataBass project.

## 1. Setup
- Install music21 if missing (commented for environments that already have it).
- Set a working directory for saving MIDI/MusicXML outputs.

In [1]:
import music21 as m21
us = m21.environment.UserSettings()

import os
import sys
import pathlib
import pandas as pd

# Récupérer WORKING_DIR
WORKING_DIR = os.getenv("WORKING_DIR")
sys.path.append(WORKING_DIR)

print("music21 settings OK")

music21 settings OK


In [2]:
# If music21 is not installed, uncomment the next line
# !pip install music21

from pathlib import Path
import music21 as m21

# Set working directory for outputs
WORKDIR = Path.cwd()
print("Working dir:", WORKDIR)
print("music21 version:", m21.__version__)

Working dir: /Users/Ith/code/Lou-SdC/DataBass/notebooks/lou
music21 version: 9.9.1


In [3]:
%load_ext autoreload
%autoreload 2

## 2. Load / Parse a Score
- Option A: Use music21 built-in corpus (e.g., Bach chorales)
- Option B: Load your own MIDI / MusicXML file
- Quickly inspect parts, measures, and notes.

In [4]:
# Option A: built-in corpus example
score = m21.corpus.parse('bach/bwv66.6')  # short Bach chorale
print(score)

# Option B: load your own file (uncomment and set path)
# score_path = Path("path/to/your/file.mid")
# score = m21.converter.parse(score_path)

# Inspect top-level structure
print("Parts:", [p.partName for p in score.parts]) # print the different
print("Measures count (part 0):", len(score.parts[0].getElementsByClass('Measure')))

# Show first measures
score.parts[0].measures(1, 5).show('text')

<music21.stream.Score bach/bwv66.6.mxl>
Parts: ['Soprano', 'Alto', 'Tenor', 'Bass']
Measures count (part 0): 10
{0.0} <music21.instrument.Instrument 'P1: Soprano: Instrument 1'>
{0.0} <music21.clef.TrebleClef>
{0.0} <music21.key.Key of f# minor>
{0.0} <music21.meter.TimeSignature 4/4>
{0.0} <music21.stream.Measure 1 offset=0.0>
    {0.0} <music21.note.Note A>
    {1.0} <music21.note.Note B>
    {2.0} <music21.note.Note C#>
    {3.0} <music21.note.Note E>
{4.0} <music21.stream.Measure 2 offset=4.0>
    {0.0} <music21.note.Note C#>
    {1.0} <music21.note.Note B>
    {2.0} <music21.note.Note A>
    {3.0} <music21.note.Note C#>
{8.0} <music21.stream.Measure 3 offset=8.0>
    {0.0} <music21.layout.SystemLayout>
    {0.0} <music21.note.Note A>
    {0.5} <music21.note.Note B>
    {1.0} <music21.note.Note G#>
    {2.0} <music21.note.Note F#>
    {3.0} <music21.note.Note A>
{12.0} <music21.stream.Measure 4 offset=12.0>
    {0.0} <music21.note.Note B>
    {1.0} <music21.note.Note B>
    {2.0} <

## 3. Extract Notes to a Table
- Flatten notes (or chords) into a list with pitch, duration, offset, part.
- Convert to pandas DataFrame for quick inspection.

In [5]:
import pandas as pd

notes_data = []
for p_idx, part in enumerate(score.parts):
    for n in part.recurse().notes:
        if n.isNote:
            pitch = n.pitch.nameWithOctave
            duration_q = n.quarterLength
            offset = float(n.offset)
            notes_data.append({
                'part': part.partName or f'Part {p_idx+1}',
                'pitch': pitch,
                'duration_q': duration_q,
                'offset_q': offset,
            })
        elif n.isChord:
            for p in n.pitches:
                duration_q = n.quarterLength
                offset = float(n.offset)
                notes_data.append({
                    'part': part.partName or f'Part {p_idx+1}',
                    'pitch': p.nameWithOctave,
                    'duration_q': duration_q,
                    'offset_q': offset,
                })

notes_df = pd.DataFrame(notes_data)
print(notes_df.head(20))
print("Total notes:", len(notes_df))

       part pitch  duration_q  offset_q
0   Soprano   C#5         0.5       0.0
1   Soprano    B4         0.5       0.5
2   Soprano    A4         1.0       0.0
3   Soprano    B4         1.0       1.0
4   Soprano   C#5         1.0       2.0
5   Soprano    E5         1.0       3.0
6   Soprano   C#5         1.0       0.0
7   Soprano    B4         1.0       1.0
8   Soprano    A4         1.0       2.0
9   Soprano   C#5         1.0       3.0
10  Soprano    A4         0.5       0.0
11  Soprano    B4         0.5       0.5
12  Soprano   G#4         1.0       1.0
13  Soprano   F#4         1.0       2.0
14  Soprano    A4         1.0       3.0
15  Soprano    B4         1.0       0.0
16  Soprano    B4         1.0       1.0
17  Soprano   F#4         1.0       2.0
18  Soprano    E4         1.0       3.0
19  Soprano    A4         1.0       0.0
Total notes: 165


## 4. Recreate a Score from a Note List
- Build a new score/part from a table of notes (pitch, duration, offset).
- Useful to validate DataBass outputs (predicted notes → partition).

In [6]:
def build_score_from_table(df: pd.DataFrame, title: str = "Recreated "):
    """
    Build a music21 Score from a DataFrame with columns: pitch, duration_q, offset_q.
    Assumes a single part; extend as needed for multiple parts.
    """
    sc = m21.stream.Score()
    sc.metadata = m21.metadata.Metadata()
    sc.metadata.title = title

    part = m21.stream.Part()
    part.partName = "Recreated Part"

    for _, row in df.iterrows():
        n = m21.note.Note(row['pitch'])
        n.quarterLength = float(row['duration_q'])
        part.insert(float(row['offset_q']), n)

    sc.insert(0, part)
    return sc

# Example: rebuild from extracted notes (single part)
recreated = build_score_from_table(notes_df[['pitch', 'duration_q', 'offset_q']])
print(recreated) #whole partition : Score object music21
print(recreated.pitches) #notes : list of note object music 21
print(recreated.offset) #start time
# Show a text preview of the first measures
recreated.measures(1,2).show('text')

<music21.stream.Score 0x125e5acf0>
[<music21.pitch.Pitch C#5>, <music21.pitch.Pitch A4>, <music21.pitch.Pitch C#5>, <music21.pitch.Pitch A4>, <music21.pitch.Pitch B4>, <music21.pitch.Pitch A4>, <music21.pitch.Pitch A4>, <music21.pitch.Pitch G#4>, <music21.pitch.Pitch F#4>, <music21.pitch.Pitch F#4>, <music21.pitch.Pitch E4>, <music21.pitch.Pitch F#4>, <music21.pitch.Pitch E4>, <music21.pitch.Pitch F#4>, <music21.pitch.Pitch F#4>, <music21.pitch.Pitch C#4>, <music21.pitch.Pitch F#4>, <music21.pitch.Pitch F#4>, <music21.pitch.Pitch C#4>, <music21.pitch.Pitch B3>, <music21.pitch.Pitch A3>, <music21.pitch.Pitch C#4>, <music21.pitch.Pitch A3>, <music21.pitch.Pitch C#4>, <music21.pitch.Pitch B3>, <music21.pitch.Pitch F#3>, <music21.pitch.Pitch D4>, <music21.pitch.Pitch D4>, <music21.pitch.Pitch F#3>, <music21.pitch.Pitch B3>, <music21.pitch.Pitch A3>, <music21.pitch.Pitch F#3>, <music21.pitch.Pitch A3>, <music21.pitch.Pitch F#3>, <music21.pitch.Pitch G#3>, <music21.pitch.Pitch F#3>, <music21

## 5. Export / Render
- Save to MIDI and MusicXML for inspection.
- Use `.show('midi')` or `.show()` (requires environment support) to hear/see output.

In [7]:
midi_path = WORKDIR / "recreated_score.mid"
xml_path = WORKDIR / "recreated_score.musicxml"

recreated.write('midi', fp=midi_path)
recreated.write('musicxml', fp=xml_path)

print("Saved:")
print("-", midi_path)
print("-", xml_path)

# Optional: play or view (may require a local MIDI player or MuseScore installed)
# recreated.show('midi')
# recreated.show()  # default viewer (MusicXML)

Saved:
- /Users/Ith/code/Lou-SdC/DataBass/notebooks/lou/recreated_score.mid
- /Users/Ith/code/Lou-SdC/DataBass/notebooks/lou/recreated_score.musicxml


## 6. Suitability Checklist for DataBass
- Can we rebuild partitions from note lists? Yes (Section 4).
- Are durations/offsets preserved? Inspect `offset_q` and `duration_q`.
- Multi-part handling: extend builder to one Part per instrument.
- Integration path: convert model outputs (note, onset, duration) → table → MusicXML/MIDI.
- Next step: add articulations/tempo/dynamics if needed.

## TESTS Lou. A partir de ce moment, le notebook va devenir bordellique

In [8]:
from databass.pipeline import MelodyReconstructor

/Users/Ith/.pyenv/versions/3.12.9/envs/DataBass/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [9]:
MR = MelodyReconstructor('conv2d')

🔧 Initializing MelodyReconstructor with CONV2D model...
Repository root: /Users/Ith/code/Lou-SdC/DataBass
Model path: /Users/Ith/code/Lou-SdC/DataBass/data/models/conv2D_model.keras
✓ Conv2D model loaded successfully


In [17]:
results = MR.run_full_pipeline(audio_file=WORKING_DIR + '/notebooks/lou/audio_cut1.wav', output_csv=WORKING_DIR + '/notebooks/lou/melody_output')


######################################################################
# DÉMARRAGE DU PIPELINE COMPLET
#====================================================================#
Audio: /Users/Ith/code/Lou-SdC/DataBass/notebooks/lou/audio_cut1.wav
Modèle: CONV2D
######################################################################


ÉTAPE 1: DÉCOUPAGE AUDIO
📁 Audio file: /Users/Ith/code/Lou-SdC/DataBass/notebooks/lou/audio_cut1.wav
📂 Output folder: notes
/Users/Ith/code/Lou-SdC/DataBass/notebooks/lou/audio_cut1.wav
True
✓ Loading audio file: /Users/Ith/code/Lou-SdC/DataBass/notebooks/lou/audio_cut1.wav (0.34 MB)
✓ Audio loaded successfully - Duration: 8.00s, Sample rate: 22050 Hz
✓ Estimated tempo: 117.45 BPM
✓ Detected 27 note(s) stored in onset_times
✓ Detected note lengths for 27 note(s)
✓ number of note lengths correspond to number of detected onsets
Folder 'notes' successfully emptied.
✓ Output folder created/verified: 'notes'
  ✓ Note 001 saved: notes/note_001.wav (278ms)
  ✓ Note 0

In [11]:
MR.results['split']['tempo']

117.45383522727273

In [12]:
test = MR.reconstruct_melody()
test


ÉTAPE 3: RECONSTRUCTION DE LA MÉLODIE
📝 Assembling 27 notes into sequence...

  [  1]    D2 |   0.01s →   0.29s ( 0.279s)
  [  2]    D2 |   0.02s →   0.25s ( 0.232s)
  [  3]   D#2 |   0.02s →   0.30s ( 0.279s)
  [  4]    E2 |   0.03s →   0.49s ( 0.464s)
  [  5]    G2 |   0.04s →   0.34s ( 0.302s)
  [  6]    G2 |   0.05s →   0.28s ( 0.232s)
  [  7]    G2 |   0.05s →   0.31s ( 0.255s)
  [  8]    E2 |   0.06s →   0.31s ( 0.255s)
  [  9]    E2 |   0.06s →   0.32s ( 0.255s)
  [ 10]   D#2 |   0.07s →   0.35s ( 0.279s)
  [ 11]    D2 |   0.08s →   0.31s ( 0.232s)
  [ 12]    D2 |   0.08s →   0.36s ( 0.279s)
  [ 13]    D2 |   0.09s →   0.37s ( 0.279s)
  [ 14]    D2 |   0.09s →   0.56s ( 0.464s)
  [ 15]    A2 |   0.11s →   0.38s ( 0.279s)
  [ 16]    D2 |   0.11s →   0.34s ( 0.232s)
  [ 17]   D#2 |   0.12s →   0.42s ( 0.302s)
  [ 18]    E2 |   0.12s →   0.57s ( 0.441s)
  [ 19]    G2 |   0.13s →   0.41s ( 0.279s)
  [ 20]    G2 |   0.14s →   0.40s ( 0.255s)
  [ 21]    G2 |   0.15s →   0.38s ( 0.232

{'success': True,
 'total_duration': np.float64(0.3338775510204081),
 'num_notes': 27,
 'melody_sequence': [{'index': 1,
   'note': np.str_('D2'),
   'start_time': np.float64(0.01020408163265306),
   'duration': np.float64(0.2786394557823129),
   'end_time': np.float64(0.288843537414966),
   'source_file': 'note_001.wav'},
  {'index': 2,
   'note': np.str_('D2'),
   'start_time': np.float64(0.016689342403628116),
   'duration': np.float64(0.23219954648526075),
   'end_time': np.float64(0.24888888888888888),
   'source_file': 'note_002.wav'},
  {'index': 3,
   'note': np.str_('D#2'),
   'start_time': np.float64(0.022086167800453516),
   'duration': np.float64(0.27863945578231286),
   'end_time': np.float64(0.3007256235827664),
   'source_file': 'note_003.wav'},
  {'index': 4,
   'note': np.str_('E2'),
   'start_time': np.float64(0.02857142857142857),
   'duration': np.float64(0.46439909297052173),
   'end_time': np.float64(0.4929705215419503),
   'source_file': 'note_004.wav'},
  {'inde

In [19]:
MR.results['reconstruction']['melody_sequence']

[{'index': 1,
  'note': np.str_('D2'),
  'start_time': np.float64(0.01020408163265306),
  'duration': np.float64(0.2786394557823129),
  'end_time': np.float64(0.288843537414966),
  'source_file': 'note_001.wav'},
 {'index': 2,
  'note': np.str_('D2'),
  'start_time': np.float64(0.016689342403628116),
  'duration': np.float64(0.23219954648526075),
  'end_time': np.float64(0.24888888888888888),
  'source_file': 'note_002.wav'},
 {'index': 3,
  'note': np.str_('D#2'),
  'start_time': np.float64(0.022086167800453516),
  'duration': np.float64(0.27863945578231286),
  'end_time': np.float64(0.3007256235827664),
  'source_file': 'note_003.wav'},
 {'index': 4,
  'note': np.str_('E2'),
  'start_time': np.float64(0.02857142857142857),
  'duration': np.float64(0.46439909297052173),
  'end_time': np.float64(0.4929705215419503),
  'source_file': 'note_004.wav'},
 {'index': 5,
  'note': np.str_('G2'),
  'start_time': np.float64(0.03931972789115646),
  'duration': np.float64(0.30185941043083897),
  '

In [14]:
# 1. Lire le CSV
#df = pd.read_csv(WORKING_DIR + "/notebooks/lou/melody_output.csv")  # Remplace par le chemin de ton CSV
df = pd.DataFrame(MR.results['reconstruction']['melody_sequence'])

# 2. Créer une partition
score = m21.stream.Score()
bpm = MR.results['split']['tempo']
seconds_per_beat = 60.0 / bpm  # Durée d'une noire en secondes

# Ajouter le tempo
metronome = m21.tempo.MetronomeMark(number=bpm)
score.append(metronome)

# Créer une partie
part = m21.stream.Part(instrumentName="Piano")
score.append(part)

# 3. Définir une résolution rythmique (ex: 16e de note)
resolution = 16  # 16e de note (4 = noire, 8 = croche, 16 = double croche)

# 4. Ajouter les notes avec arrondi des durées
for _, row in df.iterrows():
    # Convertir les durées en quarts de note
    duration = row["duration"] / seconds_per_beat
    offset = row["start_time"] / seconds_per_beat

    # Arrondir à la résolution choisie (ex: 1/16e de note)
    rounded_duration = round(duration * resolution) / resolution
    rounded_offset = round(offset * resolution) / resolution

    # Éviter les durées nulles ou négatives
    if rounded_duration <= 0:
        continue

    # Créer la note
    n = m21.note.Note(row["note"])
    n.quarterLength = rounded_duration
    n.offset = rounded_offset

    # Ajouter la note à la partie
    part.append(n)

# 4. Sauvegarder en MusicXML et MIDI
score.write('musicxml', fp='partition.xml')
score.write('midi', fp='partition.mid')
print("Partition générée sous 'partition.xml' et 'partition.mid'.")

Partition générée sous 'partition.xml' et 'partition.mid'.


In [ ]:
score.

<music21.stream.Score 0x125f7ff80>

In [15]:
MR.export_midi_xml('melody_output.mid', 'melody_output.xml')

✓ Partition générée sous melody_output.mid et melody_output.xml


True

## TEST avec Bach Cello Suite

In [45]:
import librosa
import soundfile as sf

# 1. Charger le MP3 (librosa utilise ffmpeg en arrière-plan)
audio, sr = librosa.load(WORKING_DIR + "/raw_data/songs/bach-cello-suite-ai1.mp3", sr=None)

# 2. Sauvegarder en WAV
sf.write(WORKING_DIR + "/raw_data/songs/bach-cello-suite-ai1.wav", audio, sr)

print("Fichier converti en WAV avec librosa.")

Fichier converti en WAV avec librosa.


In [46]:
results = MR.run_full_pipeline(audio_file=WORKING_DIR + '/raw_data/songs/bach-cello-suite-ai1.wav', output_csv=WORKING_DIR + '/notebooks/lou/bach_cello_output.csv')


######################################################################
# DÉMARRAGE DU PIPELINE COMPLET
#====================================================================#
Audio: /Users/Ith/code/Lou-SdC/DataBass/raw_data/songs/bach-cello-suite-ai1.wav
Modèle: CONV2D
######################################################################


ÉTAPE 1: DÉCOUPAGE AUDIO
📁 Audio file: /Users/Ith/code/Lou-SdC/DataBass/raw_data/songs/bach-cello-suite-ai1.wav
📂 Output folder: notes
/Users/Ith/code/Lou-SdC/DataBass/raw_data/songs/bach-cello-suite-ai1.wav
True
✓ Loading audio file: /Users/Ith/code/Lou-SdC/DataBass/raw_data/songs/bach-cello-suite-ai1.wav (12.75 MB)
✓ Audio loaded successfully - Duration: 151.59s, Sample rate: 44100 Hz
✓ Estimated tempo: 136.00 BPM
✓ Detected 844 note(s) stored in onset_times
✓ Detected note lengths for 844 note(s)
✓ number of note lengths correspond to number of detected onsets
✓ Output folder created/verified: 'notes'
  ✓ Note 001 saved: notes/note_001.wav (221ms

/Users/Ith/.pyenv/versions/3.12.9/envs/DataBass/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=2028
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [ 24/844] 🎵    A2 | Time:   0.05s | Duration:  0.209s | File: note_024.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [ 25/844] 🎵    E3 | Time:   0.05s | Duration:  0.221s | File: note_025.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [ 26/844] 🎵   G#2 | Time:   0.06s | Duration:  0.151s | File: note_026.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
  [ 27/844] 🎵    F1 | Time:   0.06s | Duration:  0.081s | File: note_027.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
  [ 28/844] 🎵    E1 | Time:   0.06s | Duration:  0.139s | File: note_028.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
  [ 29/844] 🎵   F#1 | Time:   0.06s | Duration:  0.035s | File: note_029.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [ 30/844] 🎵    E1 | Time:   0.06s | Duration:  0.046s | File: note_030.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
  [ 31/844] 🎵    G2 | Time:   0.06s | Duration:  0.197s | File: note_031.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
  [ 32/844] 🎵    E3 | Time:   0.06s | Dura

/Users/Ith/.pyenv/versions/3.12.9/envs/DataBass/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1543
  warnings.warn(
/Users/Ith/.pyenv/versions/3.12.9/envs/DataBass/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=2029
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
  [ 35/844] 🎵   G#2 | Time:   0.07s | Duration:  0.209s | File: note_035.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [ 36/844] 🎵    E3 | Time:   0.07s | Duration:  0.221s | File: note_036.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [ 37/844] 🎵    F1 | Time:   0.08s | Duration:  0.151s | File: note_037.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
  [ 38/844] 🎵    F1 | Time:   0.08s | Duration:  0.081s | File: note_038.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [ 39/844] 🎵    E3 | Time:   0.08s | Duration:  0.139s | File: note_039.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [ 40/844] 🎵   F#1 | Time:   0.08s | Duration:  0.035s | File: note_040.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [ 41/844] 🎵    E1 | Time:   0.08s | Duration:  0.046s | File: note_041.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
  [ 42/844] 🎵    G2 | Time:   0.08s | Duration:  0.186s | File: note_042.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [ 43/844] 🎵    F1 | Time:   0.08s | Dura

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
  [799/844] 🎵    D3 | Time:   1.55s | Duration:  0.197s | File: note_799.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [800/844] 🎵    D3 | Time:   1.55s | Duration:  0.255s | File: note_800.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
  [801/844] 🎵    D3 | Time:   1.55s | Duration:  0.186s | File: note_801.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
  [802/844] 🎵    D3 | Time:   1.56s | Duration:  0.244s | File: note_802.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
  [803/844] 🎵    D3 | Time:   1.56s | Duration:  0.197s | File: note_803.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [804/844] 🎵    C2 | Time:   1.56s | Duration:  0.163s | File: note_804.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
  [805/844] 🎵    E1 | Time:   1.56s | Duration:  0.058s | File: note_805.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
  [806/844] 🎵    D3 | Time:   1.56s | Duration:  0.209s | File: note_806.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
  [807/844] 🎵    C2 | Time:   1.57s | Dura

/Users/Ith/.pyenv/versions/3.12.9/envs/DataBass/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1543
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
  [835/844] 🎵    C3 | Time:   1.66s | Duration:  0.209s | File: note_835.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [836/844] 🎵   F#1 | Time:   1.67s | Duration:  0.035s | File: note_836.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [837/844] 🎵    D3 | Time:   1.67s | Duration:  0.209s | File: note_837.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
  [838/844] 🎵    E1 | Time:   1.67s | Duration:  0.046s | File: note_838.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [839/844] 🎵   G#2 | Time:   1.67s | Duration:  0.302s | File: note_839.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
  [840/844] 🎵    C2 | Time:   1.67s | Duration:  0.209s | File: note_840.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
  [841/844] 🎵    C3 | Time:   1.68s | Duration:  0.534s | File: note_841.wav


/Users/Ith/.pyenv/versions/3.12.9/envs/DataBass/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1544
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
  [842/844] 🎵    C3 | Time:   1.68s | Duration:  0.267s | File: note_842.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
  [843/844] 🎵    G2 | Time:   1.69s | Duration:  0.418s | File: note_843.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
  [844/844] 🎵    G2 | Time:   1.69s | Duration:  6.023s | File: note_844.wav

✓ PREDICTION SUMMARY:
  - Successful: 844/844
  - Failed: 0

ÉTAPE 3: RECONSTRUCTION DE LA MÉLODIE
📝 Assembling 844 notes into sequence...

  [  1]    G2 |   0.00s →   0.22s ( 0.221s)
  [  2]    D3 |   0.00s →   0.21s ( 0.209s)
  [  3]   F#3 |   0.01s →   0.23s ( 0.221s)
  [  4]    E1 |   0.01s →   0.24s ( 0.232s)
  [  5]    B2 |   0.01s →   0.23s ( 0.221s)
  [  6]    D3 |   0.01s →   0.22s ( 0.209s)
  [  7]    C2 |   0.02s →   0.19s ( 0.174s)
  [  8]   F#1 |   0.02s →   0.08s ( 0.058s)
  [  9]    D3 |   0.02s →   0.23s ( 0.209s)
  [ 10]    G2 |   0.02s →   0.25s ( 0.232s)
  [ 11]    D3 |   0.02s →   0.23s ( 0.209s)
  [ 12]   F#3 |   0.03s →   0.

In [53]:
# 1. Lire le CSV
df = pd.read_csv(WORKING_DIR + "/notebooks/lou/bach_cello_output.csv")  # Remplace par le chemin de ton CSV


# 2. Créer une partition
score = m21.stream.Score()
bpm = results['split_info']['tempo']
seconds_per_beat = 60.0 / bpm  # Durée d'une noire en secondes

# Ajouter le tempo
metronome = m21.tempo.MetronomeMark(number=bpm)
score.append(metronome)

# Créer une partie
part = m21.stream.Part(instrumentName="Piano")
score.append(part)

# 3. Définir une résolution rythmique (ex: 16e de note)
resolution = 16  # 16e de note (4 = noire, 8 = croche, 16 = double croche)

# 4. Ajouter les notes avec arrondi des durées
for _, row in df.iterrows():
    # Convertir les durées en quarts de note
    duration = row["duration_s"] / seconds_per_beat
    offset = row["start_time_s"] / seconds_per_beat

    # Arrondir à la résolution choisie (ex: 1/16e de note)
    rounded_duration = round(duration * resolution) / resolution
    rounded_offset = round(offset * resolution) / resolution

    # Éviter les durées nulles ou négatives
    if rounded_duration <= 0:
        continue

    # Créer la note
    n = m21.note.Note(row["note"])
    n.quarterLength = rounded_duration
    n.offset = rounded_offset

    # Ajouter la note à la partie
    part.append(n)

# 5. Sauvegarder en MusicXML et MIDI
score.write('musicxml', fp='bach_cello.xml')
score.write('midi', fp='bach_cello.mid')
print("Partition générée sous 'partition.xml' et 'partition.mid'.")

Partition générée sous 'partition.xml' et 'partition.mid'.


beam: WARNING: Found a messed up beam pair <music21.beam.Beams <music21.beam.Beam 1/start>/<music21.beam.Beam 2/partial/right>>, <music21.beam.Beams <music21.beam.Beam 1/stop>/<music21.beam.Beam 2/stop>/<music21.beam.Beam 3/partial/left>>, at index 7 of 
[<music21.beam.Beams <music21.beam.Beam 1/start>/<music21.beam.Beam 2/partial/right>>, <music21.beam.Beams <music21.beam.Beam 1/continue>>, <music21.beam.Beams <music21.beam.Beam 1/stop>>, <music21.beam.Beams <music21.beam.Beam 1/start>>, <music21.beam.Beams <music21.beam.Beam 1/stop>>, <music21.beam.Beams <music21.beam.Beam 1/start>>, <music21.beam.Beams <music21.beam.Beam 1/stop>>, <music21.beam.Beams <music21.beam.Beam 1/start>/<music21.beam.Beam 2/partial/right>>, <music21.beam.Beams <music21.beam.Beam 1/stop>/<music21.beam.Beam 2/stop>/<music21.beam.Beam 3/partial/left>>]
beam: WARNING: Found a messed up beam pair <music21.beam.Beams <music21.beam.Beam 1/start>/<music21.beam.Beam 2/partial/right>/<music21.beam.Beam 3/partial/right